In [1]:
!pip install -q datasets loralib sentencepiece
!pip uninstall transformers
!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip -q install git+https://github.com/huggingface/peft.git
!pip -q install bitsandbytes
!pip install flask-ngrok flask pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52

In [2]:
# import locale
# def getpreferredencoding(do_setlocale = True):
#     return "UTF-8"
# locale.getpreferredencoding = getpreferredencoding

!ngrok authtoken '<ngrok-token>'
!pip install -U flask-cors

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 1. Load Alpaca Model



In [3]:
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig
import textwrap


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

In [4]:
tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-7b-hf")

model = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
# model = PeftModel.from_pretrained(model, "samwit/alpaca7B-lora")
model = PeftModel.from_pretrained(model, "shadow006/logqa-alpaca-lora-7b")
# model = PeftModel.from_pretrained(model, "shadow006/alpaca7B-lora-logqa-finetuned")

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [5]:
# Alpaca QA Model
def alpaca_talk(text):
    inputs = tokenizer(
        text,
        return_tensors="pt", 
    )
    input_ids = inputs["input_ids"].cuda()

    generation_config = GenerationConfig(
        temperature=0.6,
        top_p=0.95,
        repetition_penalty=1.2,
    )
    print("Generating...")
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256,
    )
    resp_list = []
    for s in generation_output.sequences:
      resp = tokenizer.decode(s)
      resp = get_response_from_full_text(resp)
      # Remove the redundant newline characters
      resp = resp.strip()
      resp_list.append(resp)
    return resp_list[0]

# Extract the response from the full text
def get_response_from_full_text(full_text):
  return full_text.split("### Response:\n")[-1]    


def format_prompt(instruction, input):
  prompt = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
  ### Instruction: '''{instruction}'''
  ### Input: '''{input}'''
  ### Response:
  """
  return prompt

In [6]:
# Test Example
instruction = """
Help me analyze this logs and specify the causes the errors. 
"""
input = """
2023/04/02 12:28:51 /offermaker/video_user/internal/dao/user_dao.go:88 record not found
[5.951ms] [rows:0] SELECT * FROM `user` WHERE username = 'test3' ORDER BY `user`.`user_id` LIMIT 1
[GIN] 2023/04/02 - 12:28:51 | 200 |  101.488053ms |  219.143.177.52 | POST     "/users/register"
[GIN] 2023/04/02 - 13:10:41 | 404 |       2.851µs |  219.143.177.52 | GET      "/"
[GIN] 2023/04/02 - 13:10:41 | 404 |       3.604µs |  219.143.177.52 | GET      "/favicon.ico"
[GIN] 2023/04/02 - 13:16:37 | 204 |      49.559µs |  219.143.177.52 | OPTIONS  "/users/register"

2023/04/02 13:16:37 /offermaker/video_user/internal/dao/user_dao.go:83 record not found
[6.457ms] [rows:0] SELECT * FROM `user` WHERE email = 'test0402@qq.com' ORDER BY `user`.`user_id` LIMIT 1

"""
prompt = format_prompt(instruction, input)
answer = alpaca_talk(prompt)
print(answer)

Generating...
The error occurred because there was no user with the given username or email in the database, so it could not be retrieved from the database.


# 2. Rule-based extraction


In [37]:
import re
from collections import defaultdict

def analyze_all_logs(log_text):
    logs = defaultdict(list)
    pattern = r'\[(INFO|ERROR)\]\[server.go:\d+\] ([\s\S]*?)\n'
    matches = re.findall(pattern, log_text)
    for match in matches:
        logs[match[0]].append(match[1])
    result = ""
    for log_type, log_msgs in logs.items():
        result += f"{log_type}:{len(log_msgs)} in total\n"
        for i, log_msg in enumerate(log_msgs):
            result += f"{log_msg}"
            if i != len(log_msgs) - 1:
                result += "\n"
        if log_type != list(logs.keys())[-1]:
            result += "\n"
    return result

def analyze_err_logs(log_text):
    logs = defaultdict(list)
    pattern = r'\[(ERROR)\]\[server.go:\d+\] ([\s\S]*?)\n'
    matches = re.findall(pattern, log_text)
    for match in matches:
        logs[match[0]].append(match[1])
    result = ""
    for log_type, log_msgs in logs.items():
        result += f"{log_type}:{len(log_msgs)} in total\n"
        for i, log_msg in enumerate(log_msgs):
            result += f"{log_msg}"
            if i != len(log_msgs) - 1:
                result += "\n"
    return result

In [38]:
# 4. Extraction
def extract(input):
  output = analyze_all_logs(input)
  error_log = analyze_err_logs(input)
  return output, error_log

In [39]:
# test sample
log_text = """
[INFO][server.go:45] 2023/03/05 21:37:56 bind: 127.0.0.1:6399, start listening...
[INFO][server.go:77] 2023/03/05 21:39:10 accept link
[INFO][server.go:58] 2023/03/05 21:52:17 get exit signal
[INFO][server.go:62] 2023/03/05 21:52:17 shutting down...
[ERROR][server.go:72] 2023/03/05 21:52:17 listener accept err: accept tcp 127.0.0.1:6399: use of closed network connection
[INFO][server.go:108] 2023/03/05 21:52:17 handler shutting down...
[ERROR][server.go:72] 2023/03/05 22:56:56 listener accept err: accept tcp 127.0.0.1:6399: use of closed network connection
"""
output, error_log = extract(log_text)
print(output)
print(error_log)

INFO:5 in total
2023/03/05 21:37:56 bind: 127.0.0.1:6399, start listening...
2023/03/05 21:39:10 accept link
2023/03/05 21:52:17 get exit signal
2023/03/05 21:52:17 shutting down...
2023/03/05 21:52:17 handler shutting down...
ERROR:2 in total
2023/03/05 21:52:17 listener accept err: accept tcp 127.0.0.1:6399: use of closed network connection
2023/03/05 22:56:56 listener accept err: accept tcp 127.0.0.1:6399: use of closed network connection
ERROR:2 in total
2023/03/05 21:52:17 listener accept err: accept tcp 127.0.0.1:6399: use of closed network connection
2023/03/05 22:56:56 listener accept err: accept tcp 127.0.0.1:6399: use of closed network connection


# 3. Log filtering

In [50]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import os
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Download required NLTK resources
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

def extract_keywords_and_phrases(user_question):
    # Tokenize and part-of-speech tag the user question
    tokens = word_tokenize(user_question)
    # print('tokens', tokens)

    tagged_tokens = pos_tag(tokens)

    # Identify keywords and phrases based on part-of-speech tags
    keywords = []
    for i, (token, pos) in enumerate(tagged_tokens):
        if pos in ['NN', 'NNS', 'NNP', 'NNPS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
            # Nouns and verbs are likely to be relevant keywords
            keywords.append(token.lower())
    # print('pos',keywords)

    # Remove stopwords
    stopwords_list = set(stopwords.words('english'))
    keywords = [keyword for keyword in keywords if keyword not in stopwords_list]
    # print('stop',keywords)
    return keywords

def count_keywords_in_logs(logs, keywords):
    keyword_counts = []
    for log in logs:
        count = sum(1 for keyword in keywords if keyword in log.lower())
        keyword_counts.append((log, count))
    return keyword_counts

def get_logs_with_keyword_counts(logs, keywords):
    keyword_counts = count_keywords_in_logs(logs, keywords)
    logs_with_keywords = [(log, count) for log, count in keyword_counts if count > 0]
    return logs_with_keywords


# Core function
def log_filtering(log_input):
  logs = log_text.splitlines()
  log_content = '\n'.join(logs)

  user_question = "Can you analyse the logs with error of no such file or directory in the log and tell me how to solve them?"
  keywords = extract_keywords_and_phrases(user_question)
  # Initialize the TfidfVectorizer
  vectorizer = TfidfVectorizer(vocabulary=keywords)
  # Fit and transform the log content
  tfidf_matrix = vectorizer.fit_transform([log_content])
  # Convert the result to a DataFrame
  tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
  # Filter out the important keywords based on a threshold (e.g., 0.1)
  threshold = 0.1
  important_keywords = list(tfidf_df.loc[:, tfidf_df.gt(threshold).any()].columns)
  print('important:',important_keywords)

  logs_with_keyword_counts = get_logs_with_keyword_counts(logs, important_keywords)

  return_logs = []
  for log, count in logs_with_keyword_counts:
      return_logs.append(log)
      # print(log)
  return_log = '\n'.join(return_logs)
  print("filtered log:\n" + return_log)
  return return_log

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [51]:
log_input = """
[INFO][server.go:45] 2023/03/05 21:37:56 bind: 127.0.0.1:6399, start listening...
[INFO][server.go:77] 2023/03/05 21:39:10 accept link
[INFO][server.go:58] 2023/03/05 21:52:17 get exit signal
[INFO][server.go:62] 2023/03/05 21:52:17 shutting down...
[ERROR][server.go:72] 2023/03/05 21:52:17 listener accept err: accept tcp 127.0.0.1:6399: use of closed network connection
[INFO][server.go:108] 2023/03/05 21:52:17 handler shutting down...
[ERROR][server.go:72] 2023/03/05 22:56:56 listener accept err: accept tcp 127.0.0.1:6399: use of closed network connection
"""

filtered_log = log_filtering(log_input)

important: ['error']
filtered log:
[ERROR][server.go:72] 2023/03/05 21:52:17 listener accept err: accept tcp 127.0.0.1:6399: use of closed network connection
[ERROR][server.go:72] 2023/03/05 22:56:56 listener accept err: accept tcp 127.0.0.1:6399: use of closed network connection


# 4. Deployment

In [52]:
# 暴露端口，让前端可以远程调用
from flask_ngrok import run_with_ngrok
from flask import Flask, make_response, request, jsonify, current_app
import json
from flask_cors import CORS

In [53]:
from transformers.models.deta.image_processing_deta import resize
# core service functions
# 1. Question Answering
def qa_service(instruction, input):
  # log filtering before questioning
  filtered_input = log_filtering(input)
  prompt = format_prompt(instruction, filtered_input)
  return alpaca_talk(prompt)

# 2. Analyze
def analyze_service(input):
  return qa_service(analyze_instruction, input)

# 3. Report
def report_service(input):

  # Call analyze function
  analyze_res = analyze_service(input)
  # print("analyze_res: " + str(analyze_res))

  # Email format prompt: analsis result + report instruction
  prompt = format_prompt(report_instruction, analyze_res)

  return alpaca_talk(prompt)

# 4. Extraction
def extract_service(input):
  output = analyze_all_logs(input)
  error_log = analyze_err_logs(input)
  return output, error_log

In [47]:
q = "How to implement concurrent access control in a golang project?"
qa_service(q, input)

important: ['error']
Generating...


'The following code can be used for implementing concurrent access control in Golang projects:\n\npackage main {\n    import ( "net" ) // Import net package\n    \n    func ConcurrentAccessControl() *net.TCPConn{\n        c := &net.TCPConn{}\n        \n        l, _ := net.Listen("tcp", ":6399")// Listening on port 6399 and binding it with local address :6399\n      \n        if l!= nil &&!l.Close() { // Checking whether listen was successful or not\n            conn, err := l.Accept() // Accept incoming connections from client\n            \n            if err == nil { // If no error then create TCP connection object\n                c = conn                      // Assign newly created TCPConnection Object to variable \'c\'\n              \n                return c                      // Returning TCP Connection Object\n            } else { // Else print out error message\n                fmt.Println(err)              // Print Error Message\n            }\n        }else{ // Else prin

In [54]:
# default prompts

# 1. analysis function instruction
analyze_instruction = """
  Help me analyze this logs and specify the causes the errors. 
"""
# 2. report function instruction
report_instruction = """
  Help me to write a error report in an e-mail format based on the following text to the maintainer.
"""

In [11]:
# test
report_output = report_service(input)
report_output

Generating...
Generating...


'Dear Maintainer,\n    I am writing this mail to inform you about an issue we encountered while trying to retrieve data for our project. We were unable to find any record of the requested user\'s information in the database and hence the retrieval failed. The error message states "No such user exists". Please let us know if there are any steps which need to be taken by us to resolve this problem. Thank You!'

In [12]:
# run the web server
app = Flask(__name__)
app.config['CORS_HEADERS'] = 'Content-Type'
CORS(app, resources={r"/*": {"origins": "*"}})
run_with_ngrok(app)  

# use current_app to set global variable
@app.before_first_request
def set_global_variable():
  # save the input log
  current_app.log_input = """"""

  # test
  # current_app.log_input = input

# 1. upload
@app.route("/upload",methods=["POST"])
def upload():
  input = request.json.get("input")
  current_app.log_input = input
  response = make_response("success", 200)
  print("uploaded info: " + current_app.log_input)

  return response


# 2. analyse
@app.route("/analyze",methods=["POST"])
def analyze():

  if current_app.log_input is None or current_app.log_input == "":
    output = "No content detected, please upload the log."
    response = {"output": output}
    return jsonify(response)
  output = analyze_service(current_app.log_input)
  print("analyze: " + output)
  response = {"output": output}
  return jsonify(response)


# 3. question-answering
@app.route("/question-answering",methods=["POST"])
def question_answering(default_instruction=None):
  instruction = request.json.get("instruction")
  input = request.json.get("input")
  output = ""
  if default_instruction is not None:
    instruction = default_instruction

  if input is None or input == "":
    input = current_app.log_input
  # run core function
  output = qa_service(instruction, input)
  print("QA: " + output)
  response = {"output": output}
  return jsonify(response)



# 4. report: analysis result + report prompt
@app.route("/report",methods=["POST"])
def report():

  if current_app.log_input is None or current_app.log_input == "":
    print("report: " + current_app.log_input)
    output = "No content detected, please upload the log."
    response = {"output": output}
    return jsonify(response)


  output = report_service(current_app.log_input)

  print("Report: " + output)
  response = {"output": output}
  return jsonify(response)

# 5. Extraction
@app.route("/extract",methods=["POST"])
def extraction():
  output, error_log = extract_service(current_app.log_input);
  if error_log is not None:
    current_app.log_input = error_log

  response = {"output": output}
  return jsonify(response)

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://89ad-35-204-37-27.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
